<a href="https://colab.research.google.com/github/erikaduda/dsto-gan/blob/main/Pr%C3%A9_processamento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Módulo de Pré-Processamento de Dados

Este módulo do programa é responsável pelo **pré-processamento de dados**, uma etapa essencial para preparar conjuntos de dados brutos para análise ou modelagem em tarefas de aprendizado de máquina. O pré-processamento inclui a leitura, limpeza e transformação dos dados, garantindo que estejam em um formato adequado para uso em algoritmos de machine learning. A seguir, descrevemos as funcionalidades principais deste módulo:

#### 1. Leitura e Separação dos Atributos e Classe
O módulo começa lendo um arquivo de dados, que pode estar em formato CSV ou outro formato tabular. Em seguida, ele separa os **atributos** (features) da **classe** (target). Essa separação é fundamental, pois os atributos são as variáveis independentes usadas para prever a classe, que é a variável dependente ou alvo.

#### 2. Codificação de Atributos Nominais
Se os atributos forem **nominais** (categóricos não ordenados), o módulo aplica a técnica de **codificação one-hot**. Essa técnica transforma cada categoria nominal em um vetor binário, onde apenas um bit é "1" (ativo) e os demais são "0". Por exemplo, se um atributo "Cor" tiver as categorias "Vermelho", "Verde" e "Azul", a codificação one-hot criará três novas colunas binárias, uma para cada cor. Essa abordagem é necessária porque a maioria dos algoritmos de machine learning não consegue trabalhar diretamente com dados categóricos.

#### 3. Codificação da Classe Nominal
Se a classe for nominal, o módulo substitui cada item da classe por **números sequenciais**, começando por 0. Por exemplo, se a classe for "Tipo" com categorias "A", "B" e "C", ela será transformada em 0, 1 e 2, respectivamente. Essa codificação é comum em problemas de classificação, onde a classe precisa ser representada numericamente.

#### 4. Tratamento de Valores Vazios ou Nulos
O módulo verifica a presença de valores **vazios** ou **nulos** nos atributos. Se forem encontrados, ele substitui esses valores pela **média** dos valores existentes na respectiva coluna. Essa abordagem é amplamente utilizada para lidar com dados faltantes, garantindo que o conjunto de dados permaneça completo e adequado para análise. A média é calculada apenas para atributos numéricos, enquanto para atributos categóricos, pode-se utilizar a moda (valor mais frequente).

#### 5. Gravação do Resultado em Arquivo CSV
Após o pré-processamento, o módulo grava o conjunto de dados transformado em um novo arquivo **.csv**. Esse arquivo contém os dados limpos, codificados e prontos para uso em etapas subsequentes, como treinamento de modelos de machine learning. A gravação em formato CSV garante compatibilidade com a maioria das ferramentas e bibliotecas de análise de dados.

### Exemplo de Funcionamento
Suponha um arquivo de entrada com os seguintes dados:

| Idade | Cor    | Salário | Classe |
|-------|--------|---------|--------|
| 25    | Vermelho | 50000   | A      |
| 30    | Verde  |         | B      |
| 35    | Azul   | 70000   | A      |

Após o pré-processamento, o arquivo de saída seria:

| Idade | Cor_Vermelho | Cor_Verde | Cor_Azul | Salário | Classe |
|-------|--------------|-----------|----------|---------|--------|
| 25    | 1            | 0         | 0        | 50000   | 0      |
| 30    | 0            | 1         | 0        | 60000   | 1      |
| 35    | 0            | 0         | 1        | 70000   | 0      |

Neste exemplo:
- O atributo "Cor" foi codificado usando one-hot.
- O valor faltante em "Salário" foi substituído pela média (60000).
- A classe "Classe" foi codificada numericamente (A = 0, B = 1).

### Conclusão
Este módulo de pré-processamento é uma ferramenta essencial para garantir a qualidade e a consistência dos dados antes de sua utilização em modelos de machine learning. Ele automatiza tarefas como codificação de atributos categóricos, tratamento de valores faltantes e gravação de dados processados, economizando tempo e reduzindo erros manuais. Ao padronizar o pré-processamento, o módulo contribui para a criação de pipelines de dados mais robustos e eficientes, facilitando a aplicação de técnicas avançadas de análise e modelagem.

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import re
import sklearn
import os
import tensorflow as tf
import glob
from google.colab import files
from pandas import DataFrame
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from scipy.sparse import csr_matrix

# Função para aplicar codificação one-hot em atributos nominais
def apply_one_hot_encoding(data, column):
    encoded_data = pd.get_dummies(data[column], prefix=column)
    data = pd.concat([data, encoded_data], axis=1)
    data.drop(column, axis=1, inplace=True)
    return data

# Função para substituir atributos vazios pela média
def replace_missing_values(data):
    data.replace("?", np.nan, inplace=True)  # Substituir "?" por NaN
    imputer = SimpleImputer(strategy="mean")
    data_imputed = pd.DataFrame(imputer.fit_transform(data), columns=data.columns)
    return data_imputed

PATH = ("/content/drive/MyDrive/PHD_new/dataset/falta/")

csv_files = glob.glob(os.path.join(PATH, "*.csv"))
print(csv_files)
print(type(csv_files))

for path, dirs, files in os.walk(PATH):
    dirs = sorted(dirs)
    files = sorted(files)
    break

for i in range(len(files)):
    print(">>>>>>>>>>>>>>>>>>>>>>>>>>")
    print("COUNT FILE: ", i)
    print("NAME FILE:", files[i])

    # Leitura do arquivo
    try:
        data = pd.read_csv(PATH + files[i])
    except pd.errors.ParserError as e:
        print(f"Error parsing file {files[i]}: {str(e)}")
        continue

    # Separar os atributos da classe
    class_column = 'class'
    class_data = data[class_column]
    feature_data = data.drop(class_column, axis=1)

    # Verificar se os atributos são nominais e aplicar codificação one-hot se necessário
    for column in feature_data.columns:
        if feature_data[column].dtype == object:
            feature_data = apply_one_hot_encoding(feature_data, column)

    # Substituir valores "?" por média dos valores
    feature_data = replace_missing_values(feature_data)

    # Substituir classe nominal por números sequenciais ou valores binários (True=1, False=0)
    if class_data.dtype == bool:
        class_data = class_data.astype(int)  # Converte bool para int (0 e 1)
    elif class_data.dtype == object:
        class_data = class_data.map({'True': 1, 'False': 0})  # Mapeia 'True' para 1 e 'False' para 0

    # Substituir numerica por números sequenciais
    if class_data.dtype == int:
        sorted_values = sorted(class_data)
        if all(sorted_values[i] == sorted_values[0] + i for i in range(len(sorted_values))):  # Verifica se os valores são sequenciais
            class_data = class_data - sorted_values[0]  # Subtrai o primeiro valor para começar de 0
        else:
            label_encoder = LabelEncoder()
            class_data = label_encoder.fit_transform(class_data)


   # Substituir classe nominal por números sequenciais ou valores binários (True=1, False=0)
    if class_data.dtype == bool:
        class_data = class_data.astype(int)  # Converte bool para int (0 e 1)
    elif class_data.dtype == object:
        class_data = class_data.map({'True': 1, 'False': 0})  # Mapeia 'True' para 1 e 'False' para 0

    # Reunir atributos e classe novamente
    data_encoded = pd.concat([feature_data, pd.Series(class_data, name=class_column)], axis=1)

    # Gravar o resultado em um arquivo CSV
    data_encoded.to_csv('/content/drive/MyDrive/PHD_new/dataset_tratado/error/' + files[i], index=False)


['/content/drive/MyDrive/PHD_new/dataset/falta/bank-additional-full.csv']
<class 'list'>
>>>>>>>>>>>>>>>>>>>>>>>>>>
COUNT FILE:  0
NAME FILE: bank-additional-full.csv
